# 🎓 Agente de Recomendacion de Cursos 🎓

Bienvenidos al curso **Agente de Recomendacion de Cursos**. En este módulo, veremos cómo:

1. **Inicializar** un proyecto usando Azure AI Foundry.
2. **Crear un Agente** que utiliza la herramienta **BingGroundingTool** para realizar búsquedas en la web.
3. **Realizar preguntas reales** sobre temas de salud y bienestar.
4. **Recuperar y mostrar** respuestas, incluyendo enlaces a consultas de Bing y descargos de responsabilidad.

### ⚠️ Nota Importante sobre Modelos Compatibles ⚠️
> Actualmente, la funcionalidad de Bing Grounding es compatible únicamente con ciertos modelos de Azure OpenAI (por ejemplo, `gpt-4o`).
> 
> Asegúrate de utilizar un modelo compatible y de configurar el encabezado `"x-ms-enable-preview": "true"`.

## Requisitos Previos
- Haber completado el cuaderno básico de Agentes - [1-basics.ipynb](1-basics.ipynb)
- Tener configurado en Azure AI Foundry una conexión de Bing Grounding, la cual debe ser provisionada a través del portal de Azure. Consulta la documentación en ["Configuración de Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) para obtener más información.


- Se requiere un archivo `.env` en el directorio superior que contenga:
  ```bash
  PROJECT_CONNECTION_STRING=<tu-cadena-de-conexión>
  BING_CONNECTION_NAME=<nombre-de-tu-conexión-bing>
  ```

## ¡Explorando Bing Grounding!
Integraremos los resultados de búsqueda de **Bing Grounding** en nuestro agente para que pueda obtener contexto adicional desde la web. Se almacenará y mostrará el enlace de consulta de Bing, garantizando la transparencia en el proceso. 🎉


## 1. Initial Setup
Cargaremos las variables de entorno desde `.env` e inicializaremos nuestro `AIProjectClient` para gestionar agentes.

In [1]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool, MessageTextContent
from azure.core.credentials import AzureKeyCredential

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / "../.env")

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING"),
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

✅ Successfully initialized AIProjectClient


## 2. Create Bing-Grounded Agent 🌐

Recuperaremos nuestra conexión de Bing desde AI Foundry y usaremos `BingGroundingTool` para permitir que nuestro agente busque en la web. Luego crearemos un nuevo agente con descargos de responsabilidad sobre no ser médico, etc.

Asegúrate de que tu `MODEL_DEPLOYMENT_NAME` esté configurado a un modelo compatible con Bing (por ejemplo, `gpt-4o-0513`) y de añadir el encabezado    `{"x-ms-enable-preview": "true"}.`

In [6]:
def create_bing_grounded_agent():
    """Create an agent that can use Bing to ground queries with up-to-date info."""
    try:
        # 1. Retrieve Bing connection from your AI Foundry project
        bing_conn_name = os.environ.get("BING_CONNECTION_NAME")
        if not bing_conn_name:
            raise ValueError("BING_CONNECTION_NAME not set in .env")

        bing_connection = project_client.connections.get(connection_name=bing_conn_name)
        conn_id = bing_connection.id
        print(f"Bing Connection ID: {conn_id}")

        # 2. Initialize Bing grounding tool
        bing_tool = BingGroundingTool(connection_id=conn_id)

        # 3. Create an agent that can search with Bing
        agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="education-bing-agent",
            instructions="""
            Eres Virtual, un asesor virtual amigable especializado en recomendar cursos virtuales, con acceso a búsquedas en Bing.
            Recuerda siempre a los usuarios: No soy un asesor académico oficial.
            Proporciona recomendaciones claras de cursos, explica brevemente cada uno y anima a los usuarios a explorar oportunidades de aprendizaje virtual.
            
            Siempre:
            1. Proporciona descargos de responsabilidad indicando que no eres un profesional.
            2. Incentiva la consulta profesional.
            3. Utiliza Bing para referencias en tiempo real.
            4. Ofrece respuestas breves y útiles.
            """,
            tools=bing_tool.definitions,
            # Must pass special preview header to use Bing grounding (subject to change)
            headers={"x-ms-enable-preview": "true"},
        )

        print(f"🎉 Created Bing-grounded agent, ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating Bing-grounded agent: {e}")
        return None


# Create our Bing-based agent
bing_agent = create_bing_grounded_agent()

Bing Connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/lab-ai-foundry/providers/Microsoft.MachineLearningServices/workspaces/project-demo-yais/connections/hub-demo-yais-connection-BingSearch
🎉 Created Bing-grounded agent, ID: asst_kixDzvl5hQgwmO2psniXvFTq


## 3. Starting Threads & Asking Questions 💬

Crearemos hilos de conversación para cada consulta del usuario, permitiendo que el agente realice búsquedas con Bing para encontrar información relevante. Almacenaremos todos los pares `(thread, run)` en una lista para poder revisarlos en el siguiente paso.

In [7]:
bing_threads = []


def ask_bing_question(agent, user_query):
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created a conversation thread, ID: {thread.id}")

        # Post user query as a message
        user_message = project_client.agents.create_message(
            thread_id=thread.id, role="user", content=user_query
        )
        print(f"📨 Created user message with query: '{user_query}'")

        # Process the query with the agent
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id, agent_id=agent.id
        )
        print(f"🤖 Run finished with status: {run.status}\n")
        if run.last_error:
            print(f"Error detail: {run.last_error}\n")

        return thread, run
    except Exception as e:
        print(f"❌ Error asking Bing question: {e}")
        return None, None


if bing_agent:
    # We'll ask a few fun questions!
    questions = [
        "¿Cuáles son algunas nuevas tendencias en entrenamientos HIIT que debería conocer?",
        "¿Cuál es la recomendación actual de la OMS respecto a la ingesta de azúcar?",
        "¿Hay alguna novedad sobre el ayuno intermitente para el manejo del peso?",
    ]

    for q in questions:
        thr, rn = ask_bing_question(bing_agent, q)
        if thr and rn:
            bing_threads.append((thr, rn))

📝 Created a conversation thread, ID: thread_LK71jtCGOZHOjNPzZZgLwuFi
📨 Created user message with query: '¿Cuáles son algunas nuevas tendencias en entrenamientos HIIT que debería conocer?'
🤖 Run finished with status: RunStatus.COMPLETED

📝 Created a conversation thread, ID: thread_2I86VRi6RrBLrieJYFfCPElO
📨 Created user message with query: '¿Cuál es la recomendación actual de la OMS respecto a la ingesta de azúcar?'
🤖 Run finished with status: RunStatus.COMPLETED

📝 Created a conversation thread, ID: thread_6EcW8wgtol6y6n6PlhTT51q5
📨 Created user message with query: '¿Hay alguna novedad sobre el ayuno intermitente para el manejo del peso?'
🤖 Run finished with status: RunStatus.COMPLETED



## 4. Viewing Bing-Grounded Answers & Query URLs
We’ll retrieve each thread's messages, printing both the user queries and the agent's responses. We'll also fetch the run steps to display the **Bing Search Query URL** (so you can comply with the requirement to show where the data came from). You can replace `api.bing.microsoft.com` with `www.bing.com` to form a user-friendly link.

Because `RunStep` objects do **not** have `.details`, we look instead for `'request_url'` in `step["parameters"]`. If found, it's presumably the Bing step.

In [24]:
def view_bing_conversation(thread_id, run_id):
    try:
        # Print conversation
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation for thread:", thread_id)
        reversed_msgs = list(reversed(messages.data))  # oldest first
        for msg in reversed_msgs:
            role = msg.role.upper()
            if msg.content:
                for c in msg.content:
                    if hasattr(c, "text") and c.text:
                        print(f"{role}: {c.text.value}\n")

        # Retrieve run steps to get Bing search query link
        run_steps = project_client.agents.list_run_steps(
            thread_id=thread_id, run_id=run_id
        )
        steps_data = run_steps.get("data", [])
        if steps_data:
            print("\n🔎 Bing run steps:")
            for step in steps_data:
                # 'parameters' is typically where BingGroundingTool stores request_url
                params = step.get("step_details", {})
                if "tool_calls" in params:
                    # Extract Bing tool call details and get the request URL
                    tool_calls = params.get("tool_calls", [])
                    if tool_calls:
                        bing_call = tool_calls[0].get("bing_grounding", {})
                        original_url = bing_call.get("requesturl", "")
                    friendly_url = original_url.replace(
                        "api.bing.microsoft.com", "www.bing.com"
                    )
                    print(f"    Bing search URL: {friendly_url}")
        else:
            print("No run step data found for this conversation.")
    except Exception as e:
        print(f"❌ Error viewing Bing conversation: {e}")


# Display all queries and agent responses
if bing_threads:
    for thr, rn in bing_threads:
        view_bing_conversation(thr.id, rn.id)


🗣️ Conversation for thread: thread_LK71jtCGOZHOjNPzZZgLwuFi
USER: ¿Cuáles son algunas nuevas tendencias en entrenamientos HIIT que debería conocer?

ASSISTANT: Aquí tienes algunas nuevas tendencias en entrenamientos HIIT (High-Intensity Interval Training) para 2025 que deberías conocer:

1. **Tecnología Usable y Aplicaciones Móviles**: Los dispositivos portátiles y las aplicaciones móviles están tomando protagonismo, permitiendo personalizar y monitorizar en tiempo real los entrenamientos HIIT.
   
2. **Entrenamientos Híbridos**: La combinación de ejercicios de fuerza y cardiovasculares, también conocido como entrenamiento concurrente, está en auge. Este método mezcla actividades como el levantamiento de pesas con el ciclismo o la natación.
   
3. **Entrenamiento en Grupo**: Las clases de HIIT en grupo están ganando popularidad por la motivación y el compañerismo que ofrecen.

4. **Enfoque en la Salud Mental**: Los ejercicios HIIT están siendo cada vez más recomendados no solo por sus

## 5. Cleanup & Best Practices
You can optionally delete the agent once you're done. In production, you might keep it around for repeated usage.

### Best Practices
1. **Accuracy** – Bing search results may include disclaimers or partial info. Encourage verification with credible sources.
2. **Bing Query Display** – For compliance with Bing's use and display requirements, show both **website URLs** (in the agent's response) and **Bing search query URLs** (shown above). If the model includes citations, display them as well.
3. **Limits** – Keep an eye on usage, rate limits, or policy constraints for Bing.
4. **Privacy** – Filter search queries to avoid sending sensitive data.
5. **Evaluations** – Use `azure-ai-evaluation` for iterative improvement.


In [25]:
def cleanup_bing_agent(agent):
    if agent:
        try:
            project_client.agents.delete_agent(agent.id)
            print(f"🗑️ Deleted Bing-grounded agent: {agent.name}")
        except Exception as e:
            print(f"❌ Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")


# Uncomment if you want to remove the agent now
cleanup_bing_agent(bing_agent)

🗑️ Deleted Bing-grounded agent: education-bing-agent
